## TP3: Reconnaissance de signaux de communication par apprentissage profond

Students:
- Mathilde Dupouy
- Balthazar Neveu

In [43]:
# %general imports
from data_loader import get_data

from visualization import visualize_signals
from random import randint
interactive_flag = True
try:
    from ipywidgets import interact, IntSlider, fixed
    interactive_flag *= True
except:
    interactive_flag = False

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data loader
@TODO: comment on data, label_dict

In [40]:
# % data loading from disk
signals, snr, labels_id, label_dict = get_data()
data_in = [signals, snr, labels_id, label_dict]

In [42]:
# % interactive visualization
data_in = [signals, snr, labels_id, label_dict]
if interactive_flag:
    interact(
        visualize_signals,
        data_in = fixed(data_in),
        idx=IntSlider(min=0, max=signals.shape[0]-1, step=1))
else:
    for idx in range(5):
        visualize_signals(data_in = data_in, idx=randint(0, signals.shape[0]-1))

interactive(children=(IntSlider(value=0, description='idx', max=199), Output()), _dom_classes=('widget-interac…